In [0]:
%run "/Workspace/Users/sundarasandeepteja@gmail.com/E-Commerce Analytics Medallion Architecture with GenAI/config/project_config"

In [0]:
# Databricks notebook source
# ======================================
# SILVER LAYER: PRODUCT CLEANING & ENRICHMENT
# ======================================

# MAGIC %run ../config/project_config

from pyspark.sql import functions as F
from pyspark.sql.window import Window
from delta.tables import DeltaTable

print("🥈 SILVER LAYER: Product Cleaning & Enrichment")
print("=" * 60)

# ======================================
# STEP 1: READ BRONZE DATA
# ======================================
print("\n📥 Step 1: Reading Bronze products...")

bronze_products = spark.table(BRONZE_PRODUCTS_TABLE)
bronze_count = bronze_products.count()

print(f"  Bronze records: {bronze_count:,}")

# ======================================
# STEP 2: REMOVE DUPLICATES
# ======================================
print("\n🔄 Step 2: Removing duplicates...")

# Keep the most recent version of each product
window = Window.partitionBy("product_id").orderBy(F.desc("_ingested_at"))

deduped_products = bronze_products \
    .withColumn("_row_num", F.row_number().over(window)) \
    .filter(F.col("_row_num") == 1) \
    .drop("_row_num")

dup_removed = bronze_count - deduped_products.count()
print(f"  Duplicates removed: {dup_removed:,}")

# ======================================
# STEP 3: DATA VALIDATION
# ======================================
print("\n✅ Step 3: Validating data...")

# Filter invalid records
valid_products = deduped_products.filter(
    # Non-null required fields
    F.col("product_id").isNotNull() &
    F.col("product_name").isNotNull() &
    F.col("category").isNotNull() &
    
    # Valid price (positive)
    (F.col("price") > 0) &
    
    # Valid cost (positive and less than price)
    (F.col("cost") > 0) &
    (F.col("cost") < F.col("price")) &
    
    # Valid stock (non-negative)
    (F.col("stock_quantity") >= 0) &
    
    # Valid rating range (1-5)
    (F.col("avg_rating") >= 1) &
    (F.col("avg_rating") <= 5)
)

invalid_count = deduped_products.count() - valid_products.count()
print(f"  Invalid records filtered: {invalid_count:,}")

# ======================================
# STEP 4: STANDARDIZE VALUES
# ======================================
print("\n📝 Step 4: Standardizing values...")

standardized_products = valid_products \
    .withColumn("product_name", F.trim(F.col("product_name"))) \
    .withColumn("category", F.initcap(F.trim(F.col("category")))) \
    .withColumn("subcategory", F.initcap(F.trim(F.col("subcategory")))) \
    .withColumn("brand", F.initcap(F.trim(F.col("brand")))) \
    .withColumn("price", F.round(F.col("price"), 2)) \
    .withColumn("cost", F.round(F.col("cost"), 2)) \
    .withColumn("avg_rating", F.round(F.col("avg_rating"), 1))

print("  ✅ Text fields trimmed and capitalized")
print("  ✅ Numeric fields rounded")

# ======================================
# STEP 5: ADD DERIVED/ENRICHED COLUMNS
# ======================================
print("\n➕ Step 5: Adding enriched columns...")

enriched_products = standardized_products \
    .withColumn("profit_margin",
        F.round((F.col("price") - F.col("cost")) / F.col("price") * 100, 2)
    ) \
    .withColumn("profit_amount",
        F.round(F.col("price") - F.col("cost"), 2)
    ) \
    .withColumn("price_tier",
        F.when(F.col("price") < 25, "Budget")
         .when(F.col("price") < 50, "Economy")
         .when(F.col("price") < 100, "Mid-Range")
         .when(F.col("price") < 250, "Premium")
         .when(F.col("price") < 500, "Luxury")
         .otherwise("Ultra-Luxury")
    ) \
    .withColumn("margin_category",
        F.when(F.col("profit_margin") < 20, "Low Margin")
         .when(F.col("profit_margin") < 40, "Standard Margin")
         .when(F.col("profit_margin") < 60, "High Margin")
         .otherwise("Premium Margin")
    ) \
    .withColumn("rating_category",
        F.when(F.col("avg_rating") < 2.5, "Poor")
         .when(F.col("avg_rating") < 3.5, "Average")
         .when(F.col("avg_rating") < 4.0, "Good")
         .when(F.col("avg_rating") < 4.5, "Very Good")
         .otherwise("Excellent")
    ) \
    .withColumn("stock_status",
        F.when(F.col("stock_quantity") == 0, "Out of Stock")
         .when(F.col("stock_quantity") < 10, "Low Stock")
         .when(F.col("stock_quantity") < 50, "Medium Stock")
         .when(F.col("stock_quantity") < 200, "Good Stock")
         .otherwise("High Stock")
    ) \
    .withColumn("is_in_stock",
        F.when(F.col("stock_quantity") > 0, True).otherwise(False)
    ) \
    .withColumn("needs_reorder",
        F.when(F.col("stock_quantity") < 20, True).otherwise(False)
    )

print("  ✅ Profit margin calculated")
print("  ✅ Price tier assigned")
print("  ✅ Rating category assigned")
print("  ✅ Stock status determined")

# ======================================
# STEP 6: ADD SILVER METADATA
# ======================================
print("\n🏷️ Step 6: Adding Silver metadata...")

silver_products = enriched_products \
    .withColumn("_silver_processed_at", F.current_timestamp()) \
    .withColumn("_data_quality_score", F.lit(1.0)) \
    .drop("_is_processed")

# Select final columns in order
# Select final columns in order (removed "_row_hash")
final_columns = [
    # Business keys
    "product_id",
    
    # Product details
    "product_name", "category", "subcategory", "brand",
    
    # Pricing
    "price", "cost", "profit_margin", "profit_amount",
    "price_tier", "margin_category",
    
    # Ratings
    "avg_rating", "rating_category",
    
    # Inventory
    "stock_quantity", "stock_status", "is_in_stock", "needs_reorder",
    "is_available",
    
    # Metadata
    "_ingested_at", "_silver_processed_at", "_batch_id",
    "_data_quality_score"
]

silver_products_final = silver_products.select(final_columns)
print(f"  Final columns: {len(final_columns)}")

# ======================================
# STEP 7: WRITE TO SILVER
# ======================================
print("\n💾 Step 7: Writing to Silver layer...")

silver_products_final.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("category") \
    .option("overwriteSchema", "true") \
    .saveAsTable(SILVER_PRODUCTS_TABLE)

print(f"  ✅ Written to: {SILVER_PRODUCTS_TABLE}")

# Optimize
spark.sql(f"OPTIMIZE {SILVER_PRODUCTS_TABLE} ZORDER BY (brand, price_tier)")
print("  ✅ Table optimized with ZORDER on (brand, price_tier)")

# ======================================
# STEP 8: VERIFY AND SUMMARIZE
# ======================================
print("\n📊 Step 8: Verification & Summary...")

final_count = spark.table(SILVER_PRODUCTS_TABLE).count()
print(f"  Silver products: {final_count:,}")

# Price tier distribution
print("\n📊 Price Tier Distribution:")
spark.table(SILVER_PRODUCTS_TABLE) \
    .groupBy("price_tier") \
    .agg(
        F.count("*").alias("products"),
        F.round(F.avg("price"), 2).alias("avg_price"),
        F.round(F.avg("profit_margin"), 2).alias("avg_margin")
    ) \
    .orderBy(F.desc("products")) \
    .show()

# Category metrics
print("📊 Category Metrics:")
spark.table(SILVER_PRODUCTS_TABLE) \
    .groupBy("category") \
    .agg(
        F.count("*").alias("products"),
        F.round(F.avg("avg_rating"), 2).alias("avg_rating"),
        F.sum(F.when(F.col("needs_reorder"), 1).otherwise(0)).alias("needs_reorder")
    ) \
    .orderBy(F.desc("products")) \
    .show()

# Stock status
print("📊 Stock Status:")
spark.table(SILVER_PRODUCTS_TABLE) \
    .groupBy("stock_status") \
    .agg(F.count("*").alias("count")) \
    .orderBy(F.desc("count")) \
    .show()

print("\n" + "=" * 60)
print("🥈 SILVER PRODUCT CLEANING COMPLETE!")
print("=" * 60)